## Attempt to run GDM through rpy2
I never got this to work. Gave up without trying too hard and switched to 
running GDM in a notebook with an R kernel instead. Saving this in case it
is useful to try again.

In [8]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
import rpy2.robjects as robjects

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from shapely.geometry import Point
pandas2ri.activate()

# Import GDM package
gdm = importr("gdm")

# Check if it's working
#print(gdm)

In [40]:
samples = pd.read_csv("samples.csv", index_col=0)
display(samples.drop(["Longitude", "Latitude"], axis=1))
display(samples.head())
sites = pd.read_csv("sites.csv")
display(sites.head())

,Name,elevation,bio_18,bio_19,bio_8,bio_9,bio_2,bio_3,bio_1,bio_4,...,bio_6,bio_11,bio_10,bio_12,bio_13,bio_17,bio_16,bio_14,bio_15,cluster
0,Pedicularis_siphonantha,2742,222.0,127.0,6.788166,-3.835167,10.891833,46.022156,1.314104,499.83990,...,-12.24125,-5.033208,7.087791,610.0,84.0,85.0,241.0,13.0,42.186512,0
1,Pedicularis_siphonantha,4280,356.0,18.0,9.299000,-4.482917,10.830812,39.312580,2.038177,617.49180,...,-13.17575,-5.756792,9.299000,646.0,132.0,18.0,356.0,4.0,87.878700,6
2,Pedicularis_siphonantha,3785,212.0,22.0,7.581500,-5.838333,13.092271,44.395252,0.332969,609.14040,...,-16.74075,-7.140375,7.581500,408.0,78.0,19.0,212.0,5.0,73.370995,3
3,Pedicularis_siphonantha,3415,282.0,25.0,8.188875,-4.126292,11.689396,44.595165,1.677594,556.69366,...,-13.60125,-5.348084,8.188875,496.0,111.0,18.0,282.0,4.0,86.970050,10
4,Pedicularis_siphonantha,3525,245.0,12.0,8.783625,-6.090333,11.636374,41.202740,1.576771,610.07605,...,-14.62225,-6.090333,8.783625,391.0,95.0,12.0,245.0,3.0,100.388860,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,Pedicularis_oligantha,3604,341.0,41.0,11.936916,-1.399875,11.715604,43.060993,5.080094,592.03360,...,-10.61975,-2.439375,11.936916,706.0,135.0,35.0,341.0,8.0,69.364670,14
740,Pedicularis_subulatidens,3996,391.0,38.0,7.722792,-5.276333,10.716875,40.117825,0.745271,601.13900,...,-14.07325,-6.766000,7.722792,754.0,150.0,35.0,391.0,9.0,76.402120,7
741,Pedicularis_longicalyx,2804,415.0,23.0,12.597709,-1.409125,10.731874,39.233650,5.254042,624.52180,...,-9.41075,-2.699458,12.597709,751.0,151.0,22.0,415.0,6.0,87.880310,6
742,Pedicularis_dichrocephala,2579,515.0,38.0,17.521833,6.801750,10.783771,46.962020,12.847177,446.21463,...,-0.60000,6.801750,17.521833,928.0,185.0,38.0,515.0,11.0,84.734580,22


,Name,Longitude,Latitude,elevation,bio_18,bio_19,bio_8,bio_9,bio_2,bio_3,...,bio_6,bio_11,bio_10,bio_12,bio_13,bio_17,bio_16,bio_14,bio_15,cluster
0,Pedicularis_siphonantha,85.98706,28.03192,2742,222.0,127.0,6.788166,-3.835167,10.891833,46.022156,...,-12.24125,-5.033208,7.087791,610.0,84.0,85.0,241.0,13.0,42.186512,0
1,Pedicularis_siphonantha,94.67609,29.61145,4280,356.0,18.0,9.299000,-4.482917,10.830812,39.312580,...,-13.17575,-5.756792,9.299000,646.0,132.0,18.0,356.0,4.0,87.878700,6
2,Pedicularis_siphonantha,91.83506,27.92335,3785,212.0,22.0,7.581500,-5.838333,13.092271,44.395252,...,-16.74075,-7.140375,7.581500,408.0,78.0,19.0,212.0,5.0,73.370995,3
3,Pedicularis_siphonantha,88.92813,27.56507,3415,282.0,25.0,8.188875,-4.126292,11.689396,44.595165,...,-13.60125,-5.348084,8.188875,496.0,111.0,18.0,282.0,4.0,86.970050,10
4,Pedicularis_siphonantha,93.07282,28.62956,3525,245.0,12.0,8.783625,-6.090333,11.636374,41.202740,...,-14.62225,-6.090333,8.783625,391.0,95.0,12.0,245.0,3.0,100.388860,28


,cluster,bio_18,bio_19,bio_8,bio_9,bio_2,bio_3,bio_1,bio_4,bio_5,...,bio_11,bio_10,bio_12,bio_13,bio_17,bio_16,bio_14,bio_15,Longitude,Latitude
0,0,65.0,128.0,-9.771334,-0.342625,11.008479,42.421886,-3.331677,575.49524,8.20200,...,-10.467833,3.480000,343.0,51.0,38.0,130.0,8.0,47.290092,85.425532,28.507147
1,1,502.0,37.0,18.068542,3.395000,9.171833,36.411335,11.250917,590.55850,22.80400,...,3.395000,18.068542,956.0,185.0,37.0,502.0,12.0,82.424385,104.411327,27.104233
2,2,396.0,60.0,18.046959,7.656208,10.769459,47.357533,13.020291,452.38748,22.36700,...,7.017000,18.046959,851.0,143.0,58.0,396.0,13.0,64.117880,100.046441,25.914939
3,3,219.0,7.0,8.907833,-5.582250,14.557208,46.032154,1.544375,620.69280,14.96850,...,-6.322792,8.907833,321.0,85.0,7.0,219.0,2.0,111.940000,91.626168,29.038383
4,4,343.0,16.0,9.089708,-6.903750,12.766645,40.417725,0.896011,690.97974,15.26225,...,-7.782291,9.089708,581.0,129.0,16.0,343.0,4.0,95.453270,96.826735,31.349648


In [41]:
gdmTab = gdm.formatsitepair(bioData=samples, 
                         bioFormat=2, #x-y spp list
                         XColumn="Longitude", 
                         YColumn="Latitude",
                         sppColumn="Name", 
                         siteColumn="cluster", 
                         predData=sites)
head(gdmTab)

R[write to console]: Aggregation function missing: defaulting to length

R[write to console]: Error in createsitepair(dist = distData, spdata = bioData, envInfo = predData,  : 
  The length of distance values is not the same as the expected number of rows in the site-pair table, unable to proceed.

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (bioData, bioFormat, dist = "bray", abundance = FALSE,  :
R[write to console]: 
 
R[write to console]:  No abundance column was specified, so the biological data are assumed to be presences.



RRuntimeError: Error in createsitepair(dist = distData, spdata = bioData, envInfo = predData,  : 
  The length of distance values is not the same as the expected number of rows in the site-pair table, unable to proceed.


# IGNORE

In [14]:
gdf = gpd.read_file("samples.geojson")
sppTab = gdf[["Name", "cluster"]]
envTab = gpd.read_file("sites.geojson").drop("geometry", axis=1)
envTab

,cluster,bio_18,bio_19,bio_8,bio_9,bio_2,bio_3,bio_1,bio_4,bio_5,...,bio_11,bio_10,bio_12,bio_13,bio_17,bio_16,bio_14,bio_15,Longitude,Latitude
0,0,65.0,128.0,-9.771334,-0.342625,11.008479,42.421886,-3.331677,575.49524,8.202000,...,-10.467834,3.480000,343.0,51.0,38.0,130.0,8.0,47.290092,85.425533,28.507147
1,1,502.0,37.0,18.068542,3.395000,9.171833,36.411335,11.250916,590.55847,22.804001,...,3.395000,18.068542,956.0,185.0,37.0,502.0,12.0,82.424385,104.411327,27.104233
2,2,396.0,60.0,18.046959,7.656208,10.769459,47.357533,13.020291,452.38748,22.367001,...,7.017000,18.046959,851.0,143.0,58.0,396.0,13.0,64.117882,100.046441,25.914939
3,3,219.0,7.0,8.907833,-5.582250,14.557208,46.032154,1.544375,620.69281,14.968500,...,-6.322792,8.907833,321.0,85.0,7.0,219.0,2.0,111.940000,91.626168,29.038383
4,4,343.0,16.0,9.089708,-6.903750,12.766645,40.417725,0.896011,690.97974,15.262250,...,-7.782291,9.089708,581.0,129.0,16.0,343.0,4.0,95.453270,96.826735,31.349648
5,5,384.0,8.0,8.339417,-6.914834,14.581792,46.016037,1.100292,627.52344,14.186250,...,-6.914834,8.339417,617.0,147.0,8.0,384.0,1.0,103.474000,100.020824,30.101250
6,6,350.0,16.0,9.795667,-5.288333,11.241959,40.302425,2.508021,617.46057,15.029750,...,-5.288333,9.795667,633.0,130.0,16.0,350.0,4.0,88.556168,94.571977,29.640658
7,7,359.0,35.0,5.949000,-8.441000,10.707958,37.858044,-1.680417,655.13483,10.901000,...,-9.928583,5.949000,683.0,139.0,34.0,359.0,9.0,78.302277,97.486109,29.265179
8,8,373.0,18.0,10.482500,-3.928167,12.365021,43.476807,3.687031,591.92487,15.577750,...,-3.928167,10.482500,672.0,145.0,18.0,373.0,4.0,86.921410,100.166955,28.607659
9,9,366.0,16.0,9.222167,-7.466458,14.423208,41.767357,1.247479,677.82324,16.185499,...,-7.466458,9.222167,726.0,129.0,16.0,366.0,3.0,84.787041,102.371031,33.093461


In [16]:
gdmTable = gdm.formatsitepair(bioData=sppTab, 
                    bioFormat=2, #x-y spp list
                         XColumn="Longitude", 
                         YColumn="Latitude",
                         sppColumn="Name", 
                         siteColumn="cluster", 
                         predData=envTab)

R[write to console]: Aggregation function missing: defaulting to length



In [19]:
gdmTable

distance,weights,s1.xCoord,...,s2.bio_16,s2.bio_14,s2.bio_15
...,...,...,...,...,...,...


In [22]:
gdm_res = gdm(data=gdmTable, geo=True)
summary(gdm_res)

TypeError: 'InstalledSTPackage' object is not callable